Auteur : Cédric RIVET - 02/04/23

**Objectif :**

A l'aide de ce programme, il est question de récupérer un maximum de données relatives à des véhicules électriques de différentes marques. Ces données vont des caractéristiques physiques du véhicule à son autonomie, en passant par une description de la motorisation. Un effort a été fait pour récupérer les temps de recharge du véhicule suivant différents modes.

**Méthodologie :**

Au-delà du tableau comparatif proposé par le site internet ciblé, ce dernier offre des fiches techniques plus complètes. L'idée a été de procéder comme suit :
- collecte des urls des différentes fiches techniques au niveau du tableau comparatif
- collecte des différentes données pertinentes à partir des différentes fiches
- structuration de la données sous la forme d'un unique dataframe.

# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Cible

In [2]:
url_website = "https://www.fiches-auto.fr/articles-auto/voiture-electrique/s-852-comparatif-des-voitures-electriques.php#haut_menu_2018"

# Fonctions

In [3]:
def acces_au_site(site):
    
    acces_site = requests.get(site)
    if acces_site.ok :
        print(f"Le site est accessible.")
    else :
        print("Une erreur est survenue.")
        
    return acces_site.status_code

In [4]:
### Récupération des liens hypertext pointant vers les fiches techniques des VE

def liste_urls(url_website):
    
    website = requests.get(url_website)
    soup = BeautifulSoup(website.text, 'html.parser')
    list_urls = soup.select("table tbody tr td strong a")

    urls = ["https://www.fiches-auto.fr" + list_urls[i].get("href") for i in range(len(list_urls))]
    
    return urls

In [5]:
### Extraction de tout le contenu de la page

def extraction_contenu(url):
    
    contenu = requests.get(url)
    soup = BeautifulSoup(contenu.text, 'html.parser')
    
    return soup

In [6]:
### Extraction du modèle, fabricant et année

def modele_ve(soup):
    
    contenu = soup.select("h1")[0].get_text().split("Fiches technique ")[1]

    marque = contenu.split(" ")[0]
    model = contenu.split(marque+" ")[1].split(" (")[0]
    annee = contenu.split("(")[1].split(")")[0]
    
    return [marque,model,annee]

In [7]:
### Extraction du prix

def prix_ve(soup):
    
    price = soup.select("div p strong")[0].get_text()
    
    return price

In [8]:
### Extraction des données techniques

def data_table(soup, inc):
    
    focus = soup.find_all("table")[inc].select("tr td strong")
    values = [focus[i].contents[0] for i in range(len(focus))]
    
    return values

In [9]:
all_h2s = ['Puissance / Performances', 
           'Moteur électrique', 
           'Moteur électrique 2', 
           'Moteur électrique  3', 
           'Batterie / Autonomie WLTP', 
           'Temps de charge', 
           'Transmission', 
           'Dimensions / Poids', 
           'Châssis']

### La liste suivante des champs a été déterminée suite à l'analyse de l'ensemble des fiches

all_libs_rev = ['fabricant', 'modele', 'annee', 'prix', 
                'puissance_cumul', 'couple_cumul', '0_100_km_h', 'vitesse_max', 
                'position_moteur_1', 'puissance_moteur_1', 'type_moteur_1', 'couple_moteur_1', 
                'position_moteur_2', 'puissance_moteur_2', 'type_moteur_2', 'couple_moteur_2', 
                'position_moteur_3', 'puissance_moteur_3', 'type_moteur_3', 'couple_moteur_3',
                'homologation_autonomie',
                'capacite_nominale_brute', 'capacite_utile_nette', 'prise_de_charge_AC', 'prise_de_charge_DC', 'vitesse_de_charge_AC', 'vitesse_de_charge_AC_option','vitesse_de_charge_DC', 'vitesse_de_charge_DC_option','refroidissement', 
                'temps_de_charge_DC_0-80%', 'temps_de_charge_AC_0-80%_prise_dom', 'temps_de_charge_AC_0-80%_prise_dom_renforcee', 'temps_de_charge_AC_0-80%_wallbox', 'autonomie_electrique_mixte', 'conso_electrique', 'pompe_a_chaleur', '', 
                'motricite', 'rapports', 
                'longueur', 'empattement', 'hauteur', 'largeur', 'coffre', 'poids', 'coffre_avant_arriere',
                'essieu_avant', 'essieu_arriere', 'base_chassis', 'plateforme', 'pneus', 'pneus_arriere']
len(all_libs_rev)

53

In [10]:
### Temps de charge
# Les données se trouvent dans des dans balises p. 

def temps_charge(soup):

    data = [""]*4
    contenu = soup.find_all("p")
    contenu_filtre = [contenu[i].text for i in range(len(contenu)) if "Temps de charge" in contenu[i].text]

    try:
        dc = contenu_filtre[0].split("Temps de charge DC 0-80% : ")[1].split("\n")[0]
        data[0] = dc 
    except:
        pass 
        
    try:
        ac = contenu_filtre[1].split("\n\t\t\t\t")
        data[1:4] = [temps.split("Temps de charge AC 0-80% : ")[1].split(" (")[0] for temps in ac if "Temps de charge AC 0-80% : " in temps]
    except:
        pass
    
    return data

In [11]:
### Formatage des données techniques en un dataframe
# Collecte et structuration de la data avant stockage dans un dataframe unique

def formatage(soup):
    
    data = [""]*53
    
    model = modele_ve(soup)
    price = prix_ve(soup)
    
    data[0:4] = [model[0], model[1], model[2], price]
    
    contenu = soup.select("h2")
    h2s = [contenu[i].contents[0] for i in range(len(contenu))]
    
    inc = 0
    
    for h2 in h2s:

        term = h2.text

        if term == "Puissance / Performances":       
            data[4:8] = data_table(soup, inc)

        elif term == "Moteur électrique":
            data[8:12] = data_table(soup, inc)

        elif term == "Moteur électrique 2" or term == "Moteur électrique  2":
            data[12:16] = data_table(soup, inc)
                
        elif term == "Moteur électrique  3":
            data[16:20] = data_table(soup, inc)

   
        elif term == "Batterie / Autonomie WLTP": 
            data[20] = "WLTP"
            data[21:30] = data_table(soup, inc) 
            
        elif term == "Batterie / Autonomie NEDC": 
            data[20] = "NEDC"
            data[21:30] = data_table(soup, inc)
            
        elif term == "Batterie / Autonomie ": 
            data[20] = ""
            data[21:30] = data_table(soup, inc)
                

        elif term == "Temps de charge":
            data[30:34] = temps_charge(soup)            
            data[34:38] = data_table(soup, inc)

        elif term == "Transmission":
             data[38:40] = data_table(soup, inc)

        elif term == "Dimensions / Poids":
             data[40:47] = data_table(soup, inc)

        elif term == "Châssis":
             data[47:53] = data_table(soup, inc)

        else :
            #print("erreur h2 : ", h2.text)
            pass

        inc+=1
        
#    # rustine ... 
    if len(data) < 53:
        data = data + [""]*(53-len(data))
    
    df = pd.DataFrame([data], columns=all_libs_rev)
    
    return df
    

# Programme principal

In [12]:
if __name__ == "__main__":

    if acces_au_site(url_website) == 200 :
        
        print("1 - Récupération des urls")
        urls = liste_urls(url_website)
        
        print("2 - Extraction des données") 
        inc = 1
        for url in urls:

            if url == urls[0]:
                soup = extraction_contenu(url)
                df = formatage(soup)
            else:
                try:
                    soup = extraction_contenu(url)
                    df2 = formatage(soup)
                    df = pd.concat([df,df2], ignore_index=True)
                except:
                    print("erreur - n° ", inc, "->", url)
                    
            inc +=1
            
        print("3 - Stockage terminé")
        #print(df.head(3))
        
        print("4 - Sauvegarde")
        df.to_csv("ve_dataset_no_clean.csv", sep=";", encoding="utf-8", index=False)
        
        
    else:
        print("Scraping avorté")
             

Le site est accessible.
1 - Récupération des urls
2 - Extraction des données
3 - Stockage terminé
4 - Sauvegarde


In [13]:
#df.iloc[22]

In [14]:
df.shape

(164, 53)

**REMARQUE :** 
- Il sera nécessaire de nettoyer les data. Malgré les efforts réalisés au niveau de l'algorithme, le manque de rigueur et d'homogénéité dans le remplissage des fiches au niveau du site internet a engendré des erreurs dans le dataset.
- Il sera également utile de compléter les données : nationalité des fabricants, type d'homologuation pour l'autonomie, etc. 